In [1]:
import os
from datetime import datetime
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf,desc,row_number,col,year,month,hour,dayofmonth,dayofweek,to_timestamp,size,isnan,lit,date_format,to_timestamp,struct
from pyspark.sql.types import MapType, StringType, IntegerType, StructType, StructField, FloatType, ArrayType, DoubleType

In [2]:
try:
    spark
except NameError:
    spark=SparkSession.builder.appName("").getOrCreate()

In [3]:
source='cuebiq'
country='ID'

if os.getenv('CLUSTER')=='PRINCE':
    path_to_data='/scratch/spf248/covid/data' 
else:
    path_to_data='/user/spf248/covid/data'

In [23]:
pings=spark.read.parquet(os.path.join(path_to_data,source,'processed',country,'pings_geocoded','*'))
homes=spark.read.parquet(os.path.join(path_to_data,source,'processed',country,'primary_home'))
homes=homes.withColumn('latitude',col('point.latitude'))
homes=homes.withColumn('longitude',col('point.longitude'))
homes=homes.drop('point')
df=pings.join(homes,on=['cuebiq_id','latitude','longitude'],how='left')
df=df.withColumnRenamed('perfect_match','is_home')
df=df.fillna(0, subset=['is_home'])
df.write.mode("overwrite").parquet(os.path.join(path_to_data,source,'processed',country,'pings_home'))